
**I758 Wissens- und KI-basierte Systeme**

# Churn Prediction mit Neuronalen Netzwerken
 


Wir arbeiten heute mit einem weiteren "Klassiker" als Datensatz: dem IBM Customer Churn data set. Eine Dokumentation dazu können Sie [hier](https://www.ibm.com/docs/en/cognos-analytics/11.1.0?topic=samples-telco-customer-churn) finden. Es geht darum, basierend auf Kundendaten von Kunden einer (fiktiven) Telekom-Firma vorherzusagen, welcher Kunde im nächsten Monat seinen Vertrag beendet. Wir arbeiten also mit einem Klassikations-Problem: welche Kunden "churnen" (und benötigen deshalb besondere Aufmerksamkeit, z.B. einen freundlichen Anruf) und welche nicht?

Diesmal wollen wir das Verhalten mit Hilfe eines neuronalen Netzwerkes vorhersagen. Dazu nutzen wir das extrem mächtige und umfangreiche Tensorflow-Framework mit der Keras-Bibliothek.

In [ ]:
# das hier könnte länger dauern (bis zu 10 Minuten?)
%pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model

Die Daten sind online frei verfügbar, liegen der Einfachheit halber aber im Verzeichnis vor. Wir sehen Sie uns etwas im Detail an.

In [ ]:
data = pd.read_csv('data/Churn_Customers.csv')

In [ ]:
data.head()

In [ ]:
data.columns

Neuronale Netze sind von Natur aus rein numerisch arbeitende Modelle. Kategorische Variablen oder ähnliches sind nicht möglich. Versuchen wir, die Daten für unser numerisches Modell vorzubereiten. Gleich am Anfang erleben wir den ersten kleinen Stolperstein. Senior Citizen ist eine kategorische Variable - wird aber numerisch codiert. Das lässt sich (eigentlich) besser formulieren. Wir machen das Feature deshalb wieder kategorisch.

In [ ]:
data.SeniorCitizen.replace([0, 1], ["No", "Yes"], inplace= True)

Die folgende Zeile könnte Probleme machen:

In [ ]:
data.TotalCharges = data.TotalCharges.astype(float)

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 
Nanu? Was ist hier los? Finden Sie es heraus:
</div>

In [ ]:
for charge in data.TotalCharges:
    try:
        charge = float(charge)
    except:
        print("charge is: %s" % charge)

In [ ]:
for i in range(len(data)):
  if data.TotalCharges[i] == " ":
      print("Tenure is %s and Monthly charges are %s" % (data.tenure[i], data.MonthlyCharges[i]))

Beschreiben Sie hier das Problem...

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 
Was sind das für Kunden? Lassen Sie uns die Daten reparieren und nach float konvertieren.
</div>

In [ ]:
# Total Charges reparieren
...

# Total Charges konvertieren
data.TotalCharges = data.TotalCharges.astype(float)

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 
Customer ID sollte eigentlich keinen Informationsgehalt haben - dieses Feature kann weg. Schmeißen Sie es aus dem Datensatz.
</div>

In [ ]:
# ...

Das sollte für den Anfang reichen. Welche Werte bleiben übrig?

In [ ]:
for col in data.dtypes[data.dtypes == object].index:
    print(col, data[col].unique())

Auch unsere Zielfunktion kann leider nicht kategorisch sein, deshalb ersetzen wir die Werte durch Integer:

In [ ]:
data.Churn.replace(["Yes", "No"], [1, 0], inplace= True)

Jetzt nutzen wir noch das bekannte One Hot Encoding, um die restlichen kategorischen Werte zu transformieren.

In [ ]:
data = pd.get_dummies(data)

Dieser Teil sollte Ihnen sehr bekannt vorkommen. 

In [ ]:
X = data.drop("Churn", axis= 1)
y = data.Churn

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 1234)

Hier wird es spannend: so bauen wir das neuronale Netz aus mehreren Layers. Studieren Sie den Code und folgen Sie dem Ablauf bis zum Berechnen der Accuracy.

In [ ]:
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Das hier könnte etwas dauern (ein paar Minuten)

In [ ]:
model.fit(X_train, y_train, epochs=150, batch_size=10)

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
accuracy

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 
Versuchen Sie nun, dem Modell eine ganz andere Struktur (mehr/weniger Layer, größere/kleinere/andere Layer) zu geben oder das Training zu variieren. Vielleicht wollen Sie sich auch zu <a href="https://keras.io/guides/sequential_model/">Sequential Models</a> belesen. Versuchen Sie, die ursprüngliche Modell-Accuracy des Originals von 78,64 Prozent (Achtung: auf den Testdaten) zu übertreffen!
</div>

Zum Schluss können Sie Ihr Modell speichern:

In [ ]:
model.save('my_model.h5')